In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [7]:
%%time
train = pd.read_csv('../input/train5.csv')
print("train finished")

train finished
CPU times: user 51 s, sys: 4.16 s, total: 55.1 s
Wall time: 55.6 s


In [8]:
%%time
test = pd.read_csv('../input/test5.csv')
print("test finished")

test finished
CPU times: user 43.3 s, sys: 2.29 s, total: 45.6 s
Wall time: 46 s


In [9]:
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(train.shape,test.shape,sample_submission.shape)

(590540, 915) (506691, 914) (506691, 2)


In [10]:
#y = train['isFraud']
#del train['isFraud']
gc.collect()
print(train.shape,test.shape)

(590540, 915) (506691, 914)


In [ ]:
"""
importances = pd.read_csv('./importances/importances1.csv')
print(importances.shape)
importances = importances[:700]
features = [x for x in list(importances['index']) if x not in ['TransactionID','TransactionDT']]
train = train[features]
test = test[features]
"""

In [11]:
gc.collect()

0

In [12]:
print(train.shape,test.shape)

(590540, 915) (506691, 914)


In [13]:
train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_17,id_18,id_19,id_20,id_30,id_31,id_32,id_33,id_35,id_36,id_37,id_38,DeviceInfo,dayofyear,hour,dayofweek,day,card1_999999,card1_fraudulent,card2_999999,card2_fraudulent,card3_999999,card3_185.0,card4_discover,card5_nan,card5_999999,card5_137.0,card5_118.0,card5_100.0,card5_150.0,card5_229.0,card5_117.0,card5_166.0,card5_126.0,is_debit,ProductCD_C,ProductCD_W,addr1_999999,addr1_nan,addr1_888888.0,addr1_191.0,addr1_184.0,addr1_436.0,addr1_143.0,addr1_205.0,addr1_194.0,addr1_494.0,addr1_444.0,addr2_999999,addr2_nan,addr2_96.0,addr2_999999.0,TransactionAmt_decimal,TransactionAmt_check,dist1_999999,dist1_nan,dist1_91.0,dist1_130.0,dist1_116.0,dist1_108.0,dist1_86.0,dist1_95.0,dist1_114.0,dist1_33.0,dist1_12.0,dist1_27.0,dist1_50.0,dist1_55.0,dist1_54.0,dist1_42.0,dist1_47.0,dist1_35.0,dist1_43.0,dist1_49.0,dist2_999999,dist2_nan,dist2_0.0,dist2_9.0,dist2_1.0,dist2_10.0,dist2_68.0,dist2_999999.0,dist2_7.0,P_emaildomain_prefix,P_emaildomain_suffix,P_emaildomain_lesscategory,P_emaildomain_999999,P_emaildomain_nan,P_emaildomain_mail.com,P_emaildomain_outlook.es,P_emaildomain_aim.com,P_emaildomain_outlook.com,P_emaildomain_roadrunner.com,P_emaildomain_verizon.net,P_emaildomain_att.net,P_emaildomain_sbcglobal.net,P_emaildomain_rocketmail.com,P_emaildomain_windstream.net,P_emaildomain_hotmail.fr,P_emaildomain_protonmail,R_emaildomain_prefix,R_emaildomain_suffix,R_emaildomain_lesscategory,R_emaildomain_999999,R_emaildomain_nan,R_emaildomain_protonmail,R_emaildomain_outlook.com,R_emaildomain_outlook.es,R_emaildomain_icloud.com,R_emaildomain_gmail.com,R_emaildomain_bellsouth.net,R_emaildomain_sbcglobal.net,R_emaildomain_msn.com,R_emaildomain_verizon.net,R_emaildomain_hotmail.fr,R_emaildomain_att.net,group_1_mean,group_1_median,group_1_min,group_1_max,group_1_sum,group_1_std,group_1_kurtosis,group_2_mean,group_2_median,group_2_min,group_2_max,group_2_sum,group_2_std,group_2_kurtosis,group_3_mean,group_3_median,group_3_min,group_3_max,group_3_sum,group_3_std,group_3_kurtosis,group_4_mean,group_4_median,group_4_min,group_4_max,group_4_sum,group_4_std,group_4_kurt

In [15]:
cat_cols = ['ProductCD','card1','card2','card3','card4','card5','dist1','dist2','P_emaildomain','R_emaildomain','DeviceInfo']
print(len(cat_cols))

11


In [16]:
train[cat_cols].nunique()

ProductCD           5
card1            3214
card2             298
card3              10
card4               5
card5              19
dist1             499
dist2              16
P_emaildomain      60
R_emaildomain      61
DeviceInfo        187
dtype: int64

In [17]:
def convert_less_common(x):
    if x not in shared_list:
        return np.nan
    else:
        return x

In [21]:
len(list(combinations(cat_cols,3)))

165

In [23]:
for tuple_ in list(combinations(cat_cols,3)):
    f1 = tuple_[0]
    f2 = tuple_[1]
    f3 = tuple_[2]
    column_name = f1 + '_' + f2 + '_' + f3
    
    train[column_name] = train[f1].apply(str) + '_' + train[f2].apply(str) + '_' + train[f3].apply(str)
    test[column_name] = test[f1].apply(str) +  '_' +test[f2].apply(str) + '_' + test[f3].apply(str)
    
    shared_list = list( set(train[column_name]).intersection(set(test[column_name] )) )
    shared = len(shared_list)
    
    only_train = train[column_name].nunique() - len(shared_list)
    only_test = test[column_name].nunique() - len(shared_list)
    print(column_name)
    train[column_name] = train[column_name].apply(convert_less_common)
    test[column_name] = test[column_name].apply(convert_less_common)
    
    print( column_name,round(shared/(shared + only_train),3) )
    print( column_name,round(shared/(shared + only_test),3) )
    print("="*100)
    
    del shared_list,shared,only_train,only_test
    gc.collect()

ProductCD_card1_card2
ProductCD_card1_card2 0.813
ProductCD_card1_card2 0.795
ProductCD_card1_card3
ProductCD_card1_card3 0.839
ProductCD_card1_card3 0.845
ProductCD_card1_card4
ProductCD_card1_card4 0.82
ProductCD_card1_card4 0.788
ProductCD_card1_card5
ProductCD_card1_card5 0.808
ProductCD_card1_card5 0.786
ProductCD_card1_dist1
ProductCD_card1_dist1 0.536
ProductCD_card1_dist1 0.689
ProductCD_card1_dist2
ProductCD_card1_dist2 0.718
ProductCD_card1_dist2 0.689
ProductCD_card1_P_emaildomain
ProductCD_card1_P_emaildomain 0.624
ProductCD_card1_P_emaildomain 0.682
ProductCD_card1_R_emaildomain
ProductCD_card1_R_emaildomain 0.599
ProductCD_card1_R_emaildomain 0.626
ProductCD_card1_DeviceInfo
ProductCD_card1_DeviceInfo 0.544
ProductCD_card1_DeviceInfo 0.622
ProductCD_card2_card3
ProductCD_card2_card3 0.911
ProductCD_card2_card3 0.912
ProductCD_card2_card4
ProductCD_card2_card4 0.907
ProductCD_card2_card4 0.908
ProductCD_card2_card5
ProductCD_card2_card5 0.837
ProductCD_card2_card5 0.845
Pr

In [29]:
print(train.shape,test.shape)

(590540, 1209) (506691, 1208)


In [27]:
del train['card3_R_emaildomain_DeviceInfo_isnull']
gc.collect()

242

In [30]:
%%time
train.to_csv('../input/train6.csv',index=False)
test.to_csv('../input/test6.csv',index=False)

CPU times: user 15min 23s, sys: 4.62 s, total: 15min 27s
Wall time: 15min 28s


In [24]:
for tuple_ in tqdm(list(combinations(cat_cols,3))):
    f1 = tuple_[0]
    f2 = tuple_[1]
    f3 = tuple_[2]
    column_name = f1 + '_' + f2 + '_' + f3
    train[column_name + '_isnull'] = train[column_name].isnull()*1
    test[column_name + '_isnull'] = test[column_name].isnull()*1    

 78%|███████▊  | 129/165 [00:12<00:01, 29.31it/s]

MemoryError: 

 78%|███████▊  | 129/165 [00:30<00:01, 29.31it/s]

In [ ]:
for tuple_ in tqdm(list(combinations(cat_cols,3))):
    f1 = tuple_[0]
    f2 = tuple_[1]
    f3 = tuple_[1]
    column_name = f1 + '_' + f2 + '_' + f3
    train[column_name] = train[column_name].fillna('999999')
    test[column_name] = test[column_name].fillna('999999')
    le = LabelEncoder()
    le.fit(list(train[column_name]) + list(test[column_name]))
    train[column_name] = le.transform(train[column_name])
    test[column_name] = le.transform(test[column_name])
print(train.shape,test.shape)

In [ ]:
train[column_name].head()

# Training

In [ ]:
y = train['isFraud']
del train['isFraud']

In [ ]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.03,
                    'num_leaves': 2**8,
                    'max_depth':12,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':2000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 51,
                    'early_stopping_rounds':100, 
                } 

In [ ]:
gc.collect()

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
%%time
train = reduce_mem_usage(train)

In [ ]:
%%time
test = reduce_mem_usage(test)

In [ ]:
gc.collect()

In [ ]:
N = 5
kf = KFold(n_splits=N)

importance = pd.DataFrame(np.zeros((train.shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train.columns)
scores = []
y_pred = np.zeros(test.shape[0])
oof = np.zeros(train.shape[0])

for fold, (trn_idx, val_idx) in enumerate(kf.split(train, y), 1):
    print('Fold {}'.format(fold))
          
    trn_data = lgb.Dataset(train.iloc[trn_idx, :].values, label=y.iloc[trn_idx].values)
    val_data = lgb.Dataset(train.iloc[val_idx, :].values, label=y.iloc[val_idx].values)   
    
    clf = lgb.train(lgb_params, trn_data ,valid_sets=[trn_data, val_data], verbose_eval=100)

    predictions = clf.predict(train.iloc[val_idx, :].values) 
    importance.iloc[:, fold - 1] = clf.feature_importance()
    oof[val_idx] = predictions

    score = roc_auc_score(y.iloc[val_idx].values, predictions)
    scores.append(score)
    print('Fold {} ROC AUC Score {}\n'.format(fold, score))

    y_pred += clf.predict(test) / N
    
    del trn_data, val_data, predictions
    gc.collect()
    
print('Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))

In [ ]:
print(roc_auc_score(y,oof))

In [ ]:
importance['avg'] = importance.mean(axis=1)

In [ ]:
importance = importance.sort_values(by='avg',ascending=False)

In [ ]:
importance = importance.reset_index(drop=False)

In [ ]:
importance.head()

In [ ]:
importance.to_csv('../importances/importances7.csv',index=False)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission['isFraud'] = y_pred

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('../predictions/pred7.csv',index=False)

In [ ]:
!kaggle competitions submit -c ieee-fraud-detection -f ../predictions/pred7.csv -m "pred7"